This tutorial can be downloaded as part of the [Wallaroo Tutorials repository](https://github.com/WallarooLabs/Wallaroo_Tutorials/tree/main/model_uploads/xgboost-upload-tutorials).

## Wallaroo Model Upload via the Wallaroo SDK: XGBoost Classification

The following tutorial demonstrates how to upload a XGBoost Classification model to a Wallaroo instance.

The following XGBoost model types are supported by Wallaroo.  XGBoost models **not** supported by Wallaroo are supported via the [Arbitrary Python models, also known as Bring Your Own Predict (BYOP)](https://staging.docs.wallaroo.ai/wallaroo-model-operations/wallaroo-model-operations-deploy/wallaroo-model-operations-upload-register/wallaroo-model-operations-upload-register-arbitrary-python/).

| XGBoost Model Type | Wallaroo Auto Packaging Supported |
|---|---|
| XGBClassifier | √ |
| XGBRegressor | √ |
| Booster Classifier | √ |
| Booster Classifier | √ |
| Booster Regressor | √ |
| Booster Random Forest Regressor | √ |
| Booster Random Forest Classifier | √ |
| XGBRFClassifier | √ |
| XGBRFRegressor | √ |
| XGBRanker\* | X |

* XGBRanker XGBoost models are currently supported via converting them to [BYOP](https://staging.docs.wallaroo.ai/wallaroo-model-operations/wallaroo-model-operations-deploy/wallaroo-model-operations-upload-register/wallaroo-model-operations-upload-register-arbitrary-python/) models.

### Tutorial Goals

Demonstrate the following:

* Upload a XGBoost Classification model to a Wallaroo instance.
* Create a pipeline and add the model as a pipeline step.
* Perform a sample inference.

### Prerequisites

* A Wallaroo version 2023.2.1 or above instance.

### References

* [Wallaroo MLOps API Essentials Guide: Model Upload and Registrations](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-api-guide/wallaroo-mlops-api-essential-guide/wallaroo-mlops-api-essentials-guide-model-uploads/)
* [Wallaroo API Connection Guide](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-api-guide/wallaroo-mlops-connection-guide/)
* [DNS Integration Guide](https://docs.wallaroo.ai/wallaroo-operations-guide/wallaroo-configuration/wallaroo-dns-guide/)

## Tutorial Steps

### Import Libraries

The first step is to import the libraries we'll be using.  See `./requirements.txt` for a list of additional libraries used with this tutorial.

In [1]:
import json
import os
import pickle

import wallaroo
from wallaroo.pipeline import Pipeline
from wallaroo.deployment_config import DeploymentConfigBuilder
from wallaroo.object import EntityNotFoundError
from wallaroo.framework import Framework

from xgboost import XGBClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

import os
os.environ["MODELS_ENABLED"] = "true"

import pyarrow as pa
import numpy as np
import pandas as pd

### Open a Connection to Wallaroo

The next step is connect to Wallaroo through the Wallaroo client.  The Python library is included in the Wallaroo install and available through the Jupyter Hub interface provided with your Wallaroo environment.

This is accomplished using the `wallaroo.Client()` command, which provides a URL to grant the SDK permission to your specific Wallaroo environment.  When displayed, enter the URL into a browser and confirm permissions.  Store the connection into a variable that can be referenced later.

If logging into the Wallaroo instance through the internal JupyterHub service, use `wl = wallaroo.Client()`.  If logging in externally, update the `wallarooPrefix` and `wallarooSuffix` variables with the proper DNS information.  For more information on Wallaroo DNS settings, see the [Wallaroo DNS Integration Guide](https://docs.wallaroo.ai/wallaroo-operations-guide/wallaroo-configuration/wallaroo-dns-guide/).

In [2]:
wl = wallaroo.Client()

Please log into the following URL in a web browser:

	https://curly-plum-3868.keycloak.wallaroo.dev/auth/realms/master/device?user_code=GLZR-JQVZ

Login successful!


### Set Variables and Helper Functions

We'll set the name of our workspace, pipeline, models and files.  Workspace names must be unique across the Wallaroo workspace.  For this, we'll add in a randomly generated 4 characters to the workspace name to prevent collisions with other users' workspaces.  If running this tutorial, we recommend hard coding the workspace name so it will function in the same workspace each time it's run.

We'll set up some helper functions that will either use existing workspaces and pipelines, or create them if they do not already exist.

In [4]:
def get_workspace(name):
    workspace = None
    for ws in wl.list_workspaces():
        if ws.name() == name:
            workspace= ws
    if(workspace == None):
        workspace = wl.create_workspace(name)
    return workspace

def get_pipeline(name):
    try:
        pipeline = wl.pipelines_by_name(name)[0]
    except EntityNotFoundError:
        pipeline = wl.build_pipeline(name)
    return pipeline


import string
import random

# make a random 4 character suffix to prevent overwriting other user's workspaces
suffix= ''.join(random.choice(string.ascii_lowercase) for i in range(4))

suffix=''

workspace_name = f'xgboost-classification{suffix}'
pipeline_name = f'xgboost-classification'

model_name = 'xgboost-classification'
model_file_name = 'model-auto-conversion_xgboost_xgb_classification_iris.pkl'

### Create Workspace and Pipeline

We will now create the Wallaroo workspace to store our model and set it as the current workspace.  Future commands will default to this workspace for pipeline creation, model uploads, etc.  We'll create our Wallaroo pipeline to deploy our model.

In [5]:
pipeline = get_pipeline(pipeline_name)

### Configure Data Schemas

XGBoost models are uploaded to Wallaroo through the Wallaroo Client `upload_model` method.

### Upload XGBoost Model Parameters

The following parameters are required for XGBoost models.  Note that while some fields are considered as **optional** for the `upload_model` method, they are required for proper uploading of a XGBoost model to Wallaroo.

| Parameter | Type | Description |
|---|---|---|
|`name` | `string` (*Required*) | The name of the model.  Model names are unique per workspace.  Models that are uploaded with the same name are assigned as a new **version** of the model. |
|`path` | `string` (*Required*) | The path to the model file being uploaded. 
|`framework` |`string` (*Upload Method Optional, SKLearn model Required*) | Set as the `Framework.XGBOOST`. |
|`input_schema` | `pyarrow.lib.Schema` (*Upload Method Optional, SKLearn model Required*) | The input schema in Apache Arrow schema format. |
|`output_schema` | `pyarrow.lib.Schema` (*Upload Method Optional, SKLearn model Required*) | The output schema in Apache Arrow schema format. |
| `convert_wait` | `bool` (*Upload Method Optional, SKLearn model Optional*) (*Default: True*) | <ul><li>**True**: Waits in the script for the model conversion completion.</li><li>**False**:  Proceeds with the script without waiting for the model conversion process to display complete. |

Once the upload process starts, the model is containerized by the Wallaroo instance.  This process may take up to 10 minutes.

#### XGBoost Schema Inputs

XGBoost schema follows a different format than other models.  To prevent inputs from being out of order, the inputs should be submitted in a single row in the order the model is trained to accept, with **all of the data types being the same**.  If a model is originally trained to accept inputs of different data types, it will need to be retrained to only accept one data type for each column - typically `pa.float32()` is a good choice.

For example, the following DataFrame has 4 columns, each column a `float`.

|&nbsp;|sepal length (cm)|sepal width (cm)|petal length (cm)|petal width (cm)|
|---|---|---|---|---|
|0|5.1|3.5|1.4|0.2|
|1|4.9|3.0|1.4|0.2|

For submission to an XGBoost model, the data input schema will be a single array with 4 float values.

```python
input_schema = pa.schema([
    pa.field('inputs', pa.list_(pa.float32(), list_size=4))
])
```

When submitting as an inference, the DataFrame is converted to rows with the column data expressed as a single array.  The data **must** be in the same order as the model expects, which is why the data is submitted as a single array rather than JSON labeled columns:  this insures that the data is submitted in the exact order as the model is trained to accept.

Original DataFrame:

&nbsp;|sepal length (cm)|sepal width (cm)|petal length (cm)|petal width (cm)
|---|---|---|---|---|
0|5.1|3.5|1.4|0.2
1|4.9|3.0|1.4|0.2

Converted DataFrame:

|&nbsp;|inputs|
|---|---|
|0|[5.1, 3.5, 1.4, 0.2]|
|1|[4.9, 3.0, 1.4, 0.2]|

#### XGBoost Schema Outputs

Outputs for XGBoost are labeled based on the trained model outputs.  For this example, the output is both `predictions` & `probabilities`.  In the Wallaroo inference result, it is grouped with the metadata `out` as `out.output`.

```python
output_schema = pa.schema([
    pa.field('predictions', pa.int64()),
    pa.field('probabilities', pa.list_(pa.float32(), list_size=3))
])
```

```python
pipeline.infer(dataframe)
```

|&nbsp;|time|in.inputs|out.output|check_failures|
|---|---|---|---|---|
|0|2023-07-05 15:11:29.776|[5.1, 3.5, 1.4, 0.2]|0|0|
|1|2023-07-05 15:11:29.776|[4.9, 3.0, 1.4, 0.2]|0|0|


In [6]:
input_schema = pa.schema([
    pa.field('inputs', pa.list_(pa.float32(), list_size=4))
])

output_schema = pa.schema([
    pa.field('predictions', pa.int64()),
    pa.field('probabilities', pa.list_(pa.float32(), list_size=3))
])

### Upload Model

The model will be uploaded with the framework set as `Framework.XGBOOST`.

Let's load the model to inspect whether it's trained on a feature dataframe:

In [7]:
with open(model_file_name, "rb") as fp:
    raw_model = pickle.load(fp)

In [8]:
raw_model

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

Get the feature names:

In [9]:
booster = raw_model.get_booster()
original_feature_names = booster.feature_names

In [10]:
original_feature_names

['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)']

In [11]:
model = wl.upload_model(model_name, 
                        model_file_name, 
                        framework=Framework.XGBOOST, 
                        input_schema=input_schema, 
                        output_schema=output_schema)
model

Waiting for model loading - this will take up to 10.0min.
Model is pending loading to a native runtime.....
Model is attempting loading to a native runtime.successful

Ready


Name,xgboost-classification
Version,1f507986-0919-465f-aa4c-1f5624fe5cac
File Name,model-auto-conversion_xgboost_xgb_classification_iris.pkl
SHA,4a1844c460e8c8503207305fb807e3a28e788062588925021807c54ee80cc7f9
Status,ready
Image Path,None
Architecture,None
Acceleration,None
Updated At,2024-26-Mar 11:24:13


In [12]:
model.config().runtime()

'onnx'

### Deploy Pipeline

The model is uploaded and ready for use.  We'll add it as a step in our pipeline, then deploy the pipeline.  For this example we're allocated 0.25 cpu and 4 Gi RAM to the pipeline through the pipeline's deployment configuration.

In [13]:
deployment_config = DeploymentConfigBuilder() \
    .cpus(0.25).memory('1Gi') \
    .build()

In [14]:
pipeline = get_pipeline(pipeline_name)

In [15]:
# clear the pipeline if it was used before
pipeline.undeploy()
pipeline.clear()

pipeline.add_model_step(model)

pipeline.deploy(deployment_config=deployment_config)

Waiting for deployment - this will take up to 45s ............. ok


name,xgboost-classification
created,2024-03-26 11:23:21.904096+00:00
last_updated,2024-03-26 11:24:24.883984+00:00
deployed,True
arch,None
accel,None
tags,
versions,"6ab835c5-5155-441b-877b-20b2e8eafbe3, 127a189d-3260-4fa1-a396-d9817216d70a"
steps,xgboost-classification
published,False


### Run Inference

A sample inference will be run.  First the pandas DataFrame used for the inference is created, then the inference run through the pipeline's `infer` method.

In [16]:
data = pd.read_json('test-xgboost-classification-data.json')
display(data)

dataframe = pd.DataFrame({"inputs": data[:2].values.tolist()})
display(dataframe)

results = pipeline.infer(dataframe)
display(results)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2


,inputs
0,"[5.1, 3.5, 1.4, 0.2]"
1,"[4.9, 3.0, 1.4, 0.2]"


,time,in.inputs,out.predictions,out.probabilities,anomaly.count
0,2024-03-26 11:25:18.098,"[5.1, 3.5, 1.4, 0.2]",0,"[0.9814425, 0.018557489]",0
1,2024-03-26 11:25:18.098,"[4.9, 3.0, 1.4, 0.2]",0,"[0.9814425, 0.018557489]",0


### Undeploy Pipelines

With the tutorial complete, the pipeline is undeployed to return the resources back to the cluster.

In [17]:
pipeline.undeploy()

Waiting for undeployment - this will take up to 45s .................................... ok


name,xgboost-classification
created,2024-03-26 11:23:21.904096+00:00
last_updated,2024-03-26 11:24:24.883984+00:00
deployed,False
arch,None
accel,None
tags,
versions,"6ab835c5-5155-441b-877b-20b2e8eafbe3, 127a189d-3260-4fa1-a396-d9817216d70a"
steps,xgboost-classification
published,False
